In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime, os, glob, json, string, inspect, emcee
from scipy.stats import median_abs_deviation, poisson
from matplotlib.patches import Circle, Ellipse
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from astropy.io import ascii, fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from PyDynamic import interp1d_unc
from astropy.table import Table
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)
import pickle as p

## Load in the data

In [8]:
# Load in the file and image names and epoch corrections
files  = sorted(glob.glob('../../analysis/meerkat/astrometry/total_field*.fits'))

# Use the image with the largest number of sources
for f, i in zip(files[:], images[:]):
    
    # Read in the PyBDSF file parameters
    srl   = fits.open(f)
    peak  = srl[1].data['Peak_flux']
    srl.close()
    
    n_source = len(np.where(peak > 78e-6)[0])
    
# Beam major
bmajs, bmins, bpas = np.genfromtxt('../../analysis/meerkat/astrometry/beams.txt', skip_header=1).T

# Use the May 2023 images as it has the lowest RMS but the highest number of sources
good_file  = files[-2]
bmaj = bmajs[-2]
bmin = bmins[-2]
   
# Read in the PyBDSF file parameters
srl   = fits.open(good_file)
peak  = srl[1].data['Peak_flux']
isl   = srl[1].data['Isl_Total_flux']
ra    = srl[1].data['RA']
dec   = srl[1].data['DEC']
major = srl[1].data['Maj_img_plane']
minor = srl[1].data['Min_img_plane']
srl.close()
        
# Calculate offset from phase_center
phase_center = SkyCoord("18h10m44.47s -26d08m46.2s", frame='icrs')
coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame='icrs')
seps = coords.separation(phase_center).arcsec

# Sort all of the arrays by separation distance
index = np.argsort(seps)
seps  = seps[index]
peak  = peak[index]
isl   = isl[index]
ra    = ra[index]
dec   = dec[index]
major = major[index]
minor = minor[index]

# Calculate Ratios
maj_ratio  = abs(major/bmaj - 1.0)
min_ratio  = abs(minor/bmin - 1.0)
flux_ratio = abs(peak/isl   - 1.0)

# Get the number of sources from the index -- minimum included separation is 1% of image size
threshold = 0.01
seps_total      = seps[np.where((seps >= threshold * 7000) & (peak > 74.0e-6))] 
n_total_sources = np.arange(1,len(seps_total) + 1) + len(seps[np.where((seps < threshold * 7000) & (peak > 80.0e-6))])

seps_point = seps[np.where((seps >= threshold * 7000) & (peak > 74.0e-6) & (flux_ratio < 0.25) & (maj_ratio < 0.25) & (min_ratio < 0.25))]
n_point_sources = np.arange(1,len(seps_point) + 1) + len(seps[np.where((seps < threshold * 7000) & (peak > 80.0e-6) & (flux_ratio < 0.25) & (maj_ratio < 0.25) & (min_ratio < 0.25))])

# Calculate the expected number of sources in a 3 arcsecond radius
n_total_expected =  n_total_sources.astype(float)  * 3.0 ** 2  / seps_total ** 2 
n_point_expected =  n_point_sources.astype(float)  * 3.0 ** 2  / seps_point ** 2 

# Calculate the probabilities following Poissonian statistics
total_prob   = (1.0 - poisson.cdf(k = 0, mu = n_total_expected)) * 100
point_prob   = (1.0 - poisson.cdf(k = 0, mu = n_point_expected)) * 100

## Plot the probability as a function of distance from the phase center